In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [17]:
import matplotlib
import signale
import numpy as np
import matplotlib.pyplot as pl
import spectrum as sp
import pickle as pkl
import scipy.signal as scsig
import pywt
import colormaps as mycmps
import signale.tools as tools
import os
import itertools
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from sklearn.cluster import KMeans
from matplotlib.mlab import PCA as mlabPCA
from mtspec import mtspec

In [18]:
def MA(array,ord=2):
    ma = np.array([array[ii-ord/2:ii+ord/2].mean() for ii in range(array.size) if ii >= ord ],dtype=float)
    ma = np.insert(ma,0,array[0:ord/2])
    ma = np.append(ma,array[-(ord/2)-1:-1])
    return ma
def zScore(array):
    return(array - np.average(array))/ np.std(array)
def mySpecgram(sig,minFreq = None,maxFreq = None,overlap=None,
               windowSize= None,NFFT = None,Fs = None,method=None):
    if not overlap:
        overlap = 48
    kIdx = 0
    timeSlices = []
    timeCenters = []
    while kIdx < sig.size:
        #timeEdges = np.arange(0,Nbins*windowSize,windowSize)
        timeSlices.append([kIdx,kIdx+windowSize])
        kIdx += windowSize - overlap
    Pxx = []
    #print timeSlices
    for item in timeSlices:
        timeCenters.append((item[0]+np.diff(item)/2)[0])
        data = sig[item[0]:item[1]]
        if data.size < windowSize:
            ddd = windowSize - data.size
            if ddd%2 ==0:
                data = np.lib.pad(data, (ddd/2,ddd/2), 'constant', constant_values=(0,0))
            else:
                data = np.lib.pad(data, (ddd/2,ddd/2+1), 'constant', constant_values=(0,0))
        if method == 'yule':
            p = sp.pyule(data, 550, norm='biased', NFFT=NFFT,sampling=Fs)
        elif method =='fft':
            p = sp.Periodogram(data,NFFT=NFFT,sampling=Fs)
        p();
        freqz = np.array(p.frequencies())
        Pxx.append(p.psd)
    Pxx = np.array(Pxx).T
    if not minFreq:
        minFreq = freqz.min()
    if not maxFreq:
        maxFreq = freqz.max()
    indexstart = np.where(freqz >= minFreq)[0][0]
    indexend   = np.where(freqz <= maxFreq)[0][-1]
    freqz = freqz[indexstart:indexend]
    Pxx = Pxx[indexstart:indexend]
    timeCenters = np.array(timeCenters)
    return timeCenters,freqz,Pxx
def multiTaperSpecgrm(sig,minFreq = None,maxFreq = None,overlap=None,
               windowSize= None,NFFT = None,Fs = None,method=None):
    if not overlap:
        overlap = 48
    kIdx = 0
    timeSlices = []
    timeCenters = []
    while kIdx < sig.size:
        #timeEdges = np.arange(0,Nbins*windowSize,windowSize)
        timeSlices.append([kIdx,kIdx+windowSize])
        kIdx += windowSize - overlap
    Pxx = []
    for item in timeSlices: 
        timeCenters.append((item[0]+np.diff(item)/2)[0])
        data = sig[item[0]:item[1]]
        if data.size < windowSize:
            ddd = windowSize - data.size
            if ddd%2 ==0:
                data = np.lib.pad(data, (ddd/2,ddd/2), 'constant', constant_values=(0,0))
            else:
                data = np.lib.pad(data, (ddd/2,ddd/2+1), 'constant', constant_values=(0,0))
        spec, freqz = mtspec(data, 1.0, 2.0, number_of_tapers=5,quadratic=False,nfft=4096)
        freqz*=2e3
        Pxx.append(spec)
    Pxx = np.array(Pxx).T
    if not minFreq:
        minFreq = freqz.min()
    if not maxFreq:
        maxFreq = freqz.max()
    indexstart = np.where(freqz >= minFreq)[0][0]
    indexend   = np.where(freqz <= maxFreq)[0][-1]
    freqz = freqz[indexstart:indexend]
    Pxx = Pxx[indexstart:indexend]
    timeCenters = np.array(timeCenters)
    return timeCenters,freqz,Pxx

In [43]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat614Control/LinearTrack_06-01-13_Day2/'
ddayx = animalPath.split('/')[-2].split('_')[-1]
#print dday
dataFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat614Control/dataFiles/sleepDB/'

In [59]:
lfpPaths = []
for lfp in tools.locate('*-p*.lfp',animalPath):#animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
ImmoDBpaths = []
for db in tools.locate('*-p*.db',dataFolder):
    if db[1].find(ddayx)>0:
        ImmoDBpaths.append(os.path.join(db[0],db[1]))
lfpPaths = sorted(lfpPaths)
ImmoDBpaths = sorted(ImmoDBpaths)
lfpPaths,ImmoDBpaths

(['/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat614Control/LinearTrack_06-01-13_Day2/01-sleep1/CSC8-pre.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat614Control/LinearTrack_06-01-13_Day2/03-sleep2/CSC8-post.lfp'],
 ['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat614Control/dataFiles/sleepDB/ImmPeriods-Rat614Control-Day2-01-pre.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat614Control/dataFiles/sleepDB/ImmPeriods-Rat614Control-Day2-03-post.db'])

In [68]:
SleepEpochsDB = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
for item,jtem in zip(ImmoDBpaths[:],lfpPaths[:]):
    print 'LOADING %s' %jtem
    ##Loading...
    expDate = animalPath.split('/')[-2].split('_')[-1]
    session = jtem.split('/')[-2].split('-')[-1]
    session = session[:-1]+'0'+session[-1]
    animal = [prt for prt in jtem.split('/') if not prt.find('Rat')][0]
    SleepTimeIndex = pd.read_pickle(item)
    lfp = pkl.load(open(jtem,'rb'))
    t,frq,Pxx =  multiTaperSpecgrm(lfp.signal,minFreq=0.1,maxFreq=200,windowSize=2048,Fs=2000,overlap=10,NFFT=4096)
    timeInSec = t/2e3+lfp.timeAxis[0]/1e3
    #Pxx = np.clip(Pxx,0,Pxx.mean()+1.4*Pxx.std())
    pxxFlat = np.array([Pxx[:,ii]*frq for ii in range(Pxx.shape[1])]).T
    for jjj in range(pxxFlat.shape[0]):
        pxxFlat[jjj,:] = MA(pxxFlat[jjj,:],12)
    
    ##Grouping into different freqency bands, delta,theta,gamma
    Dmean = frq[frq<5].mean()
    Tmean = frq[np.logical_and(frq>6,frq<11)].mean()
    Gmean = frq[np.logical_and(frq>11,frq<50)].mean()
    Rmean = frq[frq>160].mean()
    Dbounds = np.where(frq<5)[0][[0,-1]]
    Tbounds = np.where(np.logical_and(frq>6,frq<11))[0][[0,-1]]
    Gbounds = np.where(np.logical_and(frq>11,frq<50))[0][[0,-1]]
    Rbounds = np.where(frq>160)[0][[0,-1]]
    DavgPwr = Pxx[Dbounds[0]:Dbounds[-1]+1,:].mean(axis=0)
    TavgPwr = Pxx[Tbounds[0]:Tbounds[-1]+1,:].mean(axis=0)
    GavgPwr = Pxx[Gbounds[0]:Gbounds[-1]+1,:].mean(axis=0)
    RavgPwr = Pxx[Rbounds[0]:Rbounds[-1]+1,:].mean(axis=0)
    ##################################
    DpwrNormal = pxxFlat[Dbounds[0]:Dbounds[-1]+1,:].mean(axis=0)
    TpwrNormal = pxxFlat[Tbounds[0]:Tbounds[-1]+1,:].mean(axis=0)
    GpwrNormal = pxxFlat[Gbounds[0]:Gbounds[-1]+1,:].mean(axis=0)
    RpwrNormal = pxxFlat[Rbounds[0]:Rbounds[-1]+1,:].mean(axis=0)
    ####Creating a filter for asleep times
    asleepMask = np.zeros(DavgPwr.size)
    for idx,row in SleepTimeIndex[SleepTimeIndex['type']=='asleep'].iterrows():
        asleepMask += np.logical_and(timeInSec>row.t_start,timeInSec<row.t_stop)
    ###Clustering states using k-means
    pwrData = np.array([DpwrNormal*asleepMask,TpwrNormal*asleepMask,GpwrNormal*asleepMask,RpwrNormal*asleepMask]).T
    #pwrPCA= mlabPCA(pwrData)
    if 1e2*asleepMask.sum()/asleepMask.size>90:
            Klusters   = KMeans(n_clusters=2).fit_predict(pwrData)
    else:
        Klusters   = KMeans(n_clusters=3).fit_predict(pwrData)
    #KlustersPC = KMeans(n_clusters=3).fit_predict(pwrPCA.Y)
        Klusters +=8
        awkKluster = Klusters[asleepMask==0][0]
        KlusterTags = np.array([8,9,10])
        aspKlusters = KlusterTags[KlusterTags!=awkKluster]
        Klusters[np.where(Klusters==awkKluster)[0]]-=(awkKluster-2)
        for iclust,clust in enumerate(aspKlusters):
            Klusters[np.where(Klusters==clust)[0]]-=(clust-iclust)
    print set(Klusters)
    #######################################################################################################################pl.plot(k2)
    UpIdx   = np.where(Klusters==1)[0]
    DownIdx = np.where(Klusters==0)[0]
    SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
    SingletIdxD = np.where(np.diff(UpIdx)  ==2)[0]
    while SingletIdxU.size or SingletIdxD.size:
        print SingletIdxU.size,SingletIdxD.size
        if UpIdx.size > DownIdx.size:
            SingletIdxU = np.int32(DownIdx[SingletIdxU]  + np.ones(SingletIdxU.size))
            np.put(Klusters,SingletIdxU,np.zeros(SingletIdxU.size))
            SecondFlip =False
        else:
            SingletIdxD = np.where(np.diff(UpIdx)==2)[0]
            SingletIdxD  = np.int32(UpIdx[SingletIdxD]  + np.ones(SingletIdxD.size))
            np.put(Klusters,SingletIdxD,np.ones(SingletIdxD.size))
            SecondFlip =True
        if SecondFlip:
            SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
            SingletIdxU  = np.int32(DownIdx[SingletIdxU]  + np.ones(SingletIdxU.size))
            np.put(Klusters,SingletIdxU,np.zeros(SingletIdxU.size))
        else:
            SingletIdxD = np.where(np.diff(UpIdx)==2)[0]
            SingletIdxD  = np.int32(UpIdx[SingletIdxD]  + np.ones(SingletIdxD.size))
            np.put(Klusters,SingletIdxD,np.ones(SingletIdxD.size))
        del SecondFlip
        print set(Klusters)
        ###########222#######
        ###
        ###########222#######
        K1ratio = (DpwrNormal[Klusters==1] / TpwrNormal[Klusters==1]).mean()
        K0ratio = (DpwrNormal[Klusters==0] / TpwrNormal[Klusters==0]).mean()
        K11ratio = (DavgPwr[Klusters==1] / TavgPwr[Klusters==1]).mean()
        K00ratio = (DavgPwr[Klusters==0] / TavgPwr[Klusters==0]).mean()
        print K1ratio,K0ratio
        print K11ratio,K00ratio
        if K1ratio<K0ratio:
            Ones = np.where(Klusters==1)[0]
            Zers = np.where(Klusters==0)[0] 
            print 'Cluster labels fliped!!!'
            Klusters[Ones] -= 1
            Klusters[Zers] += 1
        k1 = Klusters.copy()
        ###
        # from now on SWS episodes are Kluster ==1
        ###
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==3)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==3)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.zeros(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.zeros(dupletIdx2nd.size))
        print set(Klusters)
        ##########333############
        ###Now chunks of length 3!
        ##########333############
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==4)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==4)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        try:
            dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
            dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        except IndexError:
            print 'It\'s a bit marginal!'
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        print set(Klusters)
        ##########444############
        ###Now chunks of length 4!
        ##########444############
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==5)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        dupletIdx4th = np.int32(swsIdx[dupletIdx] +4* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        np.put(Klusters,dupletIdx4th,np.ones(dupletIdx4th.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==5)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        try:
            dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
            dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
            dupletIdx4th = np.int32(swsIdx[dupletIdx] +4* np.ones(dupletIdx.size))
        except IndexError:
            print 'It\'s a bit marginal!'
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        np.put(Klusters,dupletIdx4th,np.ones(dupletIdx4th.size))
                
        UpIdx   = np.where(Klusters==1)[0]
        DownIdx = np.where(Klusters==0)[0]
        SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
        SingletIdxD = np.where(np.diff(UpIdx)  ==2)[0]
    print 100.0*np.where(Klusters==1)[0].size/Klusters.size
    ####################################################
    SleepEpochs = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
    print set(Klusters)
    borders = np.union1d([0],np.where(pd.rolling_std(Klusters,2)>0)[0])
    Kids = ['REM','SWS','AWAKE']
    for iib,brd in enumerate(borders):
        if brd < borders[-1]:
            chunk = Klusters[brd:borders[iib+1]]
            df = pd.DataFrame([[timeInSec[brd:borders[iib+1]][0],timeInSec[brd:borders[iib+1]][-1],Kids[chunk[0]],session,expDate,animal]],
                              columns=['t0','t1','epoch','session','date','animal'])
            SleepEpochs = SleepEpochs.append(df,ignore_index=True)
        else:
            chunk = Klusters[brd:]
            df = pd.DataFrame([[timeInSec[brd:][0],timeInSec[brd:][-1],Kids[chunk[0]],session,expDate,animal]],
                             columns=['t0','t1','epoch','session','date','animal'])
            SleepEpochs = SleepEpochs.append(df,ignore_index=True)
    SleepEpochsDB = SleepEpochsDB.append(SleepEpochs,ignore_index=True)
    #############################################################################################
    f,ax=pl.subplots(1,1)
    ax.pcolormesh(timeInSec,frq,np.log10(pxxFlat),cmap='magma')
    ax.plot((lfp.timeAxis)/1e3,10*zScore(lfp.signal)+70,lw=1)
    for idx,row in SleepEpochs.iterrows():
        if row.epoch == 'SWS':
            ax.axvspan(row.t0,row.t1,ymin=0.9,ymax=0.95,color='r',alpha=0.5)
        elif row.epoch == 'REM':
            ax.axvspan(row.t0,row.t1,ymin=0.8,ymax=0.85,alpha=0.5)
    ax.plot(timeInSec,10*Klusters,lw=3)
    #############################################################################################

LOADING /home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat614Control/LinearTrack_06-01-13_Day2/01-sleep1/CSC8-pre.lfp
set([0, 1])
0 2
set([0, 1])
0.226994 0.656711
2.70332 2.66521
Cluster labels fliped!!!
set([0, 1])
set([0, 1])
94.3220338983
set([0, 1])
LOADING /home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat614Control/LinearTrack_06-01-13_Day2/03-sleep2/CSC8-post.lfp
set([0, 1])
4 1
set([0, 1])
0.62919 0.688455
2.62993 2.98555
Cluster labels fliped!!!
set([0, 1])
set([0, 1])
77.0739910314
set([0, 1])


In [69]:
SleepEpochsDB_toSave = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
SleepEpochsDB_toSave = SleepEpochsDB_toSave.append(SleepEpochsDB[SleepEpochsDB.t1-SleepEpochsDB.t0 > 10],ignore_index=True)
SleepEpochsDB_toSave.to_pickle(dataFolder+'SleepEpoch-'+animal+'-'+expDate+'.db')
SleepEpochsDB_toSave

,t0,t1,epoch,session,date,animal
0,433.207668,493.328668,SWS,sleep01,Day2,Rat614Control
1,501.480668,884.624668,SWS,sleep01,Day2,Rat614Control
2,885.643668,902.966668,REM,sleep01,Day2,Rat614Control
3,903.985668,1185.229668,SWS,sleep01,Day2,Rat614Control
4,1186.248668,1201.533668,REM,sleep01,Day2,Rat614Control
5,1202.552668,1227.008668,SWS,sleep01,Day2,Rat614Control
6,1228.027668,1239.236668,REM,sleep01,Day2,Rat614Control
7,1240.255668,1530.670668,SWS,sleep01,Day2,Rat614Control
8,1531.689668,1544.936668,REM,sleep01,Day2,Rat614Control
9,1545.955668,1634.608668,SWS,sleep01,Day2,Rat614Control


In [49]:
#SleepTimeIndex

,t_start,t_stop,type
0,2999.35511,4755.323589,asleep


In [55]:
#lfp = pkl.load(open(lfpPaths[0],'rb'))
#lfp.timeAxis

array([  432695.668,   432696.168,   432696.668, ...,  1634358.168,
        1634358.668,  1634359.168])

In [70]:
#df = pd.read_pickle(ImmoDBpaths[0])
#df.t_stop[0] = lfp.timeAxis[-100]/1e3
#df.t_start[0] = lfp.timeAxis[1]/1e3
#df.to_pickle(ImmoDBpaths[1])

In [71]:
df

,t0,t1,epoch,session,date,animal
0,4730.612668,4812.132668,SWS,sleep02,Day2,Rat614Control
